In [ ]:
import os
from openai import OpenAI

token = os.environ["GITHUB_TOKEN"]
print(token)
print(os.environ["OPENAI_API_KEY"])
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini"

client = OpenAI(
    base_url=endpoint,
    api_key=token,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "What is the capital of England?",
        }
    ],
    temperature=0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)


In [2]:
response.choices[0].message.content

'The capital of England is London.'

In [3]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown


In [45]:
file = "OutdoorClothingCatalog_1000.csv"

import pandas as pd
df = pd.read_csv(file)

df = df.iloc[:50]

# save csv file
df.to_csv("OutdoorClothingCatalog_1000_short.csv", index=False)

In [50]:
df

,Unnamed: 0,name,description
0,0,Women's Campside Oxfords,This ultracomfortable lace-to-toe Oxford boast...
1,1,"Recycled Waterhog Dog Mat, Chevron Weave",Protect your floors from spills and splashing ...
2,2,Infant and Toddler Girls' Coastal Chill Swimsu...,"She'll love the bright colors, ruffles and exc..."
3,3,"Refresh Swimwear, V-Neck Tankini Contrasts",Whether you're going for a swim or heading out...
4,4,EcoFlex 3L Storm Pants,Our new TEK O2 technology makes our four-seaso...
5,5,"Smooth Comfort Check Shirt, Slightly Fitted",Our men's slightly fitted check shirt is the p...
6,6,Mens Outdoor Research Apex Gloves,These Outdoor Research Arete gloves are perfec...
7,7,"Women's Ultraplush Sweats, Relaxed Crewneck",This crewneck sweatshirt is crafted from soft ...
8,8,Mountain Man Fleece Jacket,Our best-value fleece jacket is designed with ...
9,9,"Compact Camping Table, 25""",This aluminum portable camp table is perfect f...


In [46]:
file = "OutdoorClothingCatalog_1000_short.csv"
loader = CSVLoader(file_path=file, encoding='utf-8')


In [47]:
from langchain.indexes import VectorstoreIndexCreator

In [48]:
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings

llm = OpenAI(temperature=0.9,model=model_name)

In [49]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large",
                              dimensions=1024)

index = VectorstoreIndexCreator(
    embedding=embeddings,
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [51]:
index

VectorStoreIndexWrapper(vectorstore=<langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x00000217F00C95D0>)

In [52]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [53]:
response = index.query(query, llm)

In [54]:
display(Markdown(response))

 

| Name                                        | Description                                                                                                   |
|---------------------------------------------|---------------------------------------------------------------------------------------------------------------|
| Performance Plus Woven Shirt                | A quick-dry, breathable summer shirt made of 100% nylon with UPF 40+ sun protection, ideal for trail or city. |
| Easy-Care Classic Stripe Short-Sleeve Shirt | A light and breathable 100% cotton chambray shirt with an easy-care finish. It does not have sun protection.   |<|fim_suffix|>I don't know.<|fim_suffix|>

### step by step

In [56]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader(file_path="OutdoorClothingCatalog_1000_short.csv", encoding='utf-8')

In [58]:
docs = loader.load()

In [59]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000_short.csv', 'row': 0}, page_content="Unnamed: 0: 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [60]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)

C:\Users\m.landoulsi\AppData\Local\Temp\ipykernel_13792\917072604.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)
c:\Users\m.landoulsi\Desktop\pprojects\langchain\venv\Lib\site-packages\langchain_community\embeddings\openai.py:271: UserWarning: WARNING! dimensions is not default parameter.
                    dimensions was transferred to model_kwargs.
                    Please confirm that dimensions is what you intended.
  warnings.warn(


In [76]:
embed = embeddings.embed_query("Hi folks, I am Nathan!")

In [77]:
len(embed)

1024

In [87]:
db = DocArrayInMemorySearch.from_documents(embedding=embeddings,
                            documents=docs)

In [88]:
db

In [89]:
query = "Please suggest a shirt."

In [91]:
docs = db.similarity_search(query)

In [92]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000_short.csv', 'row': 14}, page_content="Unnamed: 0: 14\nname: Easy-Care Classic Stripe Short-Sleeve Shirt, Traditional Fit\ndescription: The ultimate striped chambray shirt – we kept everything you love and made it even better. Light and breathable in 100% cotton, with an easy-care finish that sheds wrinkles remarkably well. \n\nSize & Fit\nTraditional Fit: Relaxed through the chest, sleeve and waist. \n\nWhy We Love It\nEven when something is loved as much as a classic chambray, there's always something you can do to make it even better. And that's just what we did – introducing our exclusive easy-care chambray. It's light, breathable and sheds wrinkles. Just throw it in the wash, and it's ready to wear right out of the dryer. \n\nFabric & Care\n100% cotton chambray. A light iron gives it an even crisper look. Machine wash and dry. \n\nAdditional Features\nSingle patch pocket. Button-down collar. Imported. \n\nQuestions?\nFeel fr

In [93]:
len(docs)

4

In [94]:
retriever = db.as_retriever()

In [95]:
llm = OpenAI(temperature=0,model=model_name)

In [97]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [98]:
qdocs

"Unnamed: 0: 14\nname: Easy-Care Classic Stripe Short-Sleeve Shirt, Traditional Fit\ndescription: The ultimate striped chambray shirt – we kept everything you love and made it even better. Light and breathable in 100% cotton, with an easy-care finish that sheds wrinkles remarkably well. \n\nSize & Fit\nTraditional Fit: Relaxed through the chest, sleeve and waist. \n\nWhy We Love It\nEven when something is loved as much as a classic chambray, there's always something you can do to make it even better. And that's just what we did – introducing our exclusive easy-care chambray. It's light, breathable and sheds wrinkles. Just throw it in the wash, and it's ready to wear right out of the dryer. \n\nFabric & Care\n100% cotton chambray. A light iron gives it an even crisper look. Machine wash and dry. \n\nAdditional Features\nSingle patch pocket. Button-down collar. Imported. \n\nQuestions?\nFeel free to contact us with any questions or inquiries.Unnamed: 0: 5\nname: Smooth Comfort Check Shir

In [100]:
response = llm.invoke(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 


In [101]:
display(Markdown(response))

 

Answer:

| Name | Description |
| --- | --- |
| Easy-Care Classic Stripe Short-Sleeve Shirt, Traditional Fit | A light and breathable 100% cotton chambray shirt with an easy-care finish that sheds wrinkles. Traditional fit, relaxed through the chest, sleeve, and waist. |
| Smooth Comfort Check Shirt, Slightly Fitted | A slightly fitted check shirt made of 100% cotton poplin with wrinkle-free performance. Features TrueCool® fabric that wicks moisture away and dries quickly. |
| Men's Maine Explorer Zip-Front Jac-Shirt | A classic wool jac-shirt that combines a jacket and a shirt, perfect for layering. Made of 85% wool and 15% nylon, it is warm and durable. |
| Rangeley Camp Performance Checked Flannel Shirt | A rugged flannel shirt made of 70% cotton and 30% polyester, designed for performance with moisture-wicking properties and abrasion-resistant construction. |<|fim_suffix|>| Name | Description |
| --- | --- |
| Easy-Care Classic Stripe Short-Sleeve Shirt, Traditional Fit | A light and breathable 100% cotton chambray shirt with an easy-care finish that sheds wrinkles. Traditional fit, relaxed through the chest, sleeve, and waist. |
|

In [103]:
qa_stuff = RetrievalQA.from_chain_type(retriever=retriever, llm=llm,chain_type="stuff",verbose=True)

In [105]:
qa_stuff.invoke("Please suggest a shirt.")



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Please suggest a shirt.',
 'result': " I suggest the Easy-Care Classic Stripe Short-Sleeve Shirt, Traditional Fit. It is made of 100% cotton, light and breathable, and has an easy-care finish that sheds wrinkles remarkably well. It is perfect for casual wear and can be worn right out of the dryer.<|fim_suffix|>I suggest the Smooth Comfort Check Shirt, Slightly Fitted. It is made of 100% cotton poplin with wrinkle-free performance, and it features TrueCool® fabric that wicks moisture away from your skin. This shirt is slightly fitted, providing a relaxed fit through the chest and sleeve with a slightly slimmer waist, making it a great choice for your wardrobe.<|fim_suffix|><|fim_suffix|>I suggest the Men's Maine Explorer Zip-Front Jac-Shirt. This classic wool jac-shirt is perfect for layering in colder weather and can also be worn as an outer shirt when it's warm. It features a relaxed fit, two welted handwarmer pockets, and an attached exterior wool cape for added warmth and

In [106]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

In [107]:
response = index.query(query, llm=llm)

In [108]:
display(Markdown(response))

 I suggest the Easy-Care Classic Stripe Short-Sleeve Shirt, Traditional Fit. It is light, breathable, and has an easy-care finish that sheds wrinkles remarkably well. It is made of 100% cotton and is perfect for casual wear.<|fim_suffix|>I suggest the Smooth Comfort Check Shirt, Slightly Fitted. It is made of 100% cotton poplin with wrinkle-free performance and features TrueCool® fabric that wicks moisture away from your skin. It has a slightly fitted design, making it a great choice for a comfortable and stylish look.<|fim_suffix|>I suggest the Men's Maine Explorer Zip-Front Jac-Shirt. It is a versatile piece that combines the features of a jacket and a shirt, making it suitable for year-round wear. It is made of a durable wool blend and is perfect for layering in colder weather. Plus, it has multiple pockets for added functionality.<|fim_suffix|>I suggest the Rangeley Camp Performance Checked Flannel Shirt. It is designed for the adventurous type, made with a blend of cotton and polyester for warmth and comfort. It features moisture-wicking properties, reinforced elbows, and side-seam pockets, making it both practical and stylish for outdoor activities.<|fim_suffix|>I don't know.<|fim_suffix|>I don't know.<|fim_suffix|>I don't know.<|fim_suffix|>I don't know.